In [17]:
#Imports
import pandas as pd
import altair as alt
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
sns.set_theme(style="ticks")
import matplotlib.pyplot as plt

In [18]:
data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')

testClean = data
testClean["job"] = testClean["job"].replace(['unknown'], "admin.")
testClean["marital"] = testClean["marital"].replace(['unknown'], "married")
testClean["education"] = testClean["education"].replace(['unknown'], "university.degree")
testClean["default"] = testClean["default"].replace(['unknown'], "no")
testClean["housing"] = testClean["housing"].replace(['unknown'], "yes")
testClean["loan"] = testClean["loan"].replace(['unknown'], "no")
testClean['poutcome'] = testClean['poutcome'].replace(['nonexistent'], "failure")
testClean['pdays'] = testClean['pdays'].apply(lambda x: 0 if x == 999 else x)
testClean[['job','marital',"education",'default','housing','contact','month','day_of_week','poutcome','loan','y']] = testClean[['job','marital',"education",'default','housing','contact','month','day_of_week','poutcome','loan','y']].apply(lambda x: pd.factorize(x)[0])
testClean = testClean[~testClean['pdays'].isna()]
testClean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37069 entries, 0 to 37068
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             37069 non-null  int64  
 1   job             37069 non-null  int64  
 2   marital         37069 non-null  int64  
 3   education       37069 non-null  int64  
 4   default         37069 non-null  int64  
 5   housing         37069 non-null  int64  
 6   loan            37069 non-null  int64  
 7   contact         37069 non-null  int64  
 8   month           37069 non-null  int64  
 9   day_of_week     37069 non-null  int64  
 10  campaign        37069 non-null  int64  
 11  pdays           37069 non-null  int64  
 12  previous        37069 non-null  int64  
 13  poutcome        37069 non-null  int64  
 14  emp.var.rate    37069 non-null  float64
 15  cons.price.idx  37069 non-null  float64
 16  cons.conf.idx   37069 non-null  float64
 17  euribor3m       37069 non-null 

In [19]:
X = testClean.drop('y', axis=1)
y = testClean['y']

In [20]:
mutual_info = mutual_info_classif(X, y)
mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info.sort_values(ascending=False)

euribor3m         0.074700
cons.conf.idx     0.069349
cons.price.idx    0.068907
nr.employed       0.064713
emp.var.rate      0.059807
pdays             0.030937
poutcome          0.028651
month             0.025326
previous          0.019553
contact           0.013763
job               0.009967
age               0.009898
campaign          0.005461
education         0.005332
housing           0.002345
day_of_week       0.001102
marital           0.000990
default           0.000260
loan              0.000000
dtype: float64

In [21]:
sel_three_feat = SelectKBest(mutual_info_classif, k=9).fit(X, y)
sel_bool = sel_three_feat.get_support()
X_sel = X[X.columns[sel_bool]]

In [22]:
X_main, X_test, y_main, y_test = train_test_split(X_sel, y, test_size=0.1 , random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.2, random_state=42)

In [23]:
ros = RandomOverSampler(random_state=42)
X_train_balanced, y_train_balanced = ros.fit_resample(X_train, y_train)

In [24]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [34]:
clf = RandomForestClassifier(random_state=25, n_jobs=-1)
scorers = make_scorer(recall_score)
grid_search = GridSearchCV(clf, param_grid, scoring= scorers, refit='recall_score', cv=5, return_train_score=True, n_jobs=-1)

In [35]:
grid_search.fit(X_train_balanced, y_train_balanced)
#print("Best hyperparameters: ", grid_search.best_params_)


GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=25),
             n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             refit='recall_score', return_train_score=True,
             scoring=make_scorer(recall_score))

In [36]:
print("Accuracy score: ", grid_search.best_score_)

Accuracy score:  0.6562336521447529


In [37]:
clf = RandomForestClassifier(n_estimators=grid_search.best_params_['n_estimators'],
                              max_depth=grid_search.best_params_['max_depth'],
                              min_samples_split=grid_search.best_params_['min_samples_split'],
                              min_samples_leaf=grid_search.best_params_['min_samples_leaf'],
                              random_state=25, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)



In [39]:
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.90      0.98      0.94      3251
           1       0.63      0.23      0.34       456

    accuracy                           0.89      3707
   macro avg       0.77      0.61      0.64      3707
weighted avg       0.87      0.89      0.87      3707



In [ ]:
conmat = confusion_matrix(y_test, y_pred)
print(conmat)

[[3167   84]
 [ 345  111]]
